# Reinforcement learning - Assignment 1

Course: Data Mining for Networks

Author: Nicolas Arrieta Larraza

Date: 17-02-2021

## GYM exercise

### Initializating

#### Importing libraries and necessary packages

In [ ]:
!pip install cmake 'gym[atari]' scipy
import gym.spaces
import numpy as np
import matplotlib.pyplot as plt
env = gym.make("Taxi-v3")

#### Setting up state, action and Q-table matrices

In [ ]:
state_space = env.observation_space.n
action_space = env.action_space.n
qtable = np.zeros((state_space,action_space))
print("Q-table initialized with zeros and with dimension:",qtable.shape)

Q-table initialized with zeros and with dimension: (500, 6)


### Training algorithm

Let's first define the hyperparameters of the algorithm

In [ ]:
epsilon = 1.0 #Max greed
epsilon_min = 0.005 #Min greed
epsilon_decay = 0.99994 #Rate of epsilon decay after each episode
episodes = 50000 #Number of games
max_steps = 100 #Max steps per episide
lr = 0.65 #Learning rate 
gamma = 0.65 #Discount factor (1 for long-term rewards, 0 for immediate rewards) 

Definiton of complete Q-learning training algorithm

In [ ]:
def q_learning_algorithm(epsilon, epsilon_min, epsilon_decay, episodes, max_steps, lr, gamma):

  for episode in range(episodes):
    #Reset the game parameters
    state = env.reset()
    done = False
    score = 0

    for _ in range(max_steps):

      #Take best action in Q-table (exploitation)
      if np.random.uniform(0,1) > epsilon:
        action = np.argmax(qtable[state,:])
      #Take random action (exploration)
      else:
        action = env.action_space.sample()

      #Take a step
      next_state, reward, done, _ = env.step(action)

      #add up score
      score += reward

      #Update Q-table with new Q value
      qtable[state,action] = (1-lr)*qtable[state,action] + lr*(reward+gamma*np.max(qtable[next_state,:]))

      #Update state
      state = next_state

      if done:
        break

    #Reducing epsilon each episode (exploration-exploitation trade-off)
    if epsilon >= epsilon_min:
      epsilon += epsilon_decay

  

Let's compute the training algorithm

In [ ]:
q_learning_algorithm(epsilon, epsilon_min, epsilon_decay, episodes, max_steps, lr, gamma)

### Testing algorithm

Let's compare the performance of the Q-learning algorithm vs a simple brute-force algoritm

In [ ]:
def test_q_learning(episodes, max_steps):

  total_epochs, total_penalties = 0, 0

  for episode in range(episodes):
    #Reset the game parameters
    state = env.reset()
    done = False
    score = 0
    epochs, penalties = 0, 0

    for _ in range(max_steps):

      action = np.argmax(qtable[state,:])

      #Take a step
      next_state, reward, done, _ = env.step(action)

      #add up score
      score += reward

      #Update state
      state = next_state

      epochs+=1

      if reward < 0:
        penalties+=1

      if done:
        break
    
    total_penalties += penalties
    total_epochs += epochs
  
  print(f"Results after {episodes} episodes:")
  print(f"Average timesteps per episode: {total_epochs / episodes}")
  print(f"Average penalties per episode: {total_penalties / episodes}")

In [ ]:
def test_simple_algorithm(episodes, max_steps):

  total_epochs, total_penalties = 0, 0
  
  for episode in range(episodes):
    state = env.reset()
    done = False
    score = 0
    epochs, penalties = 0, 0

    for _ in range(max_steps):
      #Take random action
      action = env.action_space.sample()

      #Take a step
      next_state, reward, done, _ = env.step(action)

      #add up score
      score += reward

      #Update state
      state = next_state

      epochs+=1

      if reward < 0:
        penalties+=1

      if done:
        break
        
    total_penalties += penalties
    total_epochs += epochs
  
  print(f"Results after {episodes} episodes:")
  print(f"Average timesteps per episode: {total_epochs / episodes}")
  print(f"Average penalties per episode: {total_penalties / episodes}")

Definition of testing parameters

In [ ]:
test_episodes = 500
test_max_steps = 100

In [ ]:
test_q_learning(test_episodes, test_max_steps)

Results after 500 episodes:
Average timesteps per episode: 13.044
Average penalties per episode: 12.044


In [ ]:
test_simple_algorithm(test_episodes, test_max_steps)

Results after 500 episodes:
Average timesteps per episode: 99.762
Average penalties per episode: 99.748


In the light of the results one can observe the efectivity of Q-learning against a brute-force algorithm:


*   The brute force algorithm takes **~86 more steps**, meaning an increase of **664.81%**
*   The brute force algorithm commits **~88 more penalties** meaning an increase of **728.2%**

